<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/2_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🔗 **Aprendizaje por Reglas de Asociación: Algoritmo Apriori**

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1.  **Comprender la intuición** detrás de las reglas de asociación y sus métricas clave: **Soporte, Confianza y Lift**.
2.  **Realizar el preprocesamiento de datos** necesario para transformar datos transaccionales en un formato analizable.
3.  **Aplicar el algoritmo Apriori** utilizando Python para extraer reglas de asociación significativas de un dataset real.
4.  **Interpretar los resultados** del modelo en un contexto de inteligencia de mercados para proponer decisiones de negocio accionables.

## Introducción

Imaginemos que somos analistas de datos en Alkosto. Nuestro objetivo es simple: entender qué productos suelen comprar juntos nuestros clientes. Si podemos descifrar estos patrones, podemos tomar decisiones más inteligentes sobre marketing, ofertas y cómo organizar los productos en la tienda.

El Análisis de Reglas de Asociación es la técnica que nos permite encontrar estas conexiones ocultas en los datos de ventas. Para ello, usamos tres métricas clave que nos ayudan a medir qué tan fuerte es una "regla" (como "si los clientes compran pan, también compran leche").

1. **Soporte (Support):** *¿Qué tan popular es un producto?*

- **Pregunta de Negocio:** ¿Con qué frecuencia se compran ciertos productos o combinaciones de productos?

- **Explicación Directa:** El soporte es simplemente el porcentaje de todas las transacciones (o cestas de compra) que contienen un producto o una combinación específica. Nos dice qué tan popular es algo.

- **Fórmula:**

$$\text{Soporte}(A \cup B) = \frac{\text{Numero de transacciones con A y B}}{\text{Total de transacciones}}*100$$

- **Ejemplo:**
  - Si de 1.000 carritos de compra, 300 llevan pan, el soporte del pan es 300/1000 = 30%.
  - Si de esos 1000 carritos, 100 llevan pan y leche juntos, el soporte de la combinación {pan, leche} es 100/1000 = 10%.

2. **Confianza (Confidence):** *¿Cuál es la asociación?*

- **Pregunta de Negocio:** Si un cliente ya ha puesto el producto A en su cesta, ¿qué tan probable es que también ponga el producto B?

- **Explicación Directa:** La confianza mide la probabilidad de que se compre un producto B, dado que ya se compró un producto A. Mide la fiabilidad de la inferencia {A} -> {B}.

- **Fórmula:**

$$\text{Confianza}(A \rightarrow B) = \frac{\text{Soporte}(A \cup B)}{\text{Soporte}(A)}*100$$

- **Ejemplo:**

  - Consideremos la regla {pan} -> {leche}. Ya sabemos que 300 carritos tienen pan. Si de esos 300, 100 también tienen leche, la confianza de la regla es 100/300 = 33% (siguiendo la fórmula: 10%/30% = 33%). Esto significa que el 33% de las veces que alguien compra pan, también compra leche.

  Analicemos la fórmula:

  Confianza = ((Número de transacciones con pan y leche) / (Total de transacciones)) / ((Número de transacciones con pan) / (Total de transacciones))
  
  Se anula (Total de transacciones) y queda:

  Confianza = (Número de transacciones con pan y leche) / (Número de transacciones con pan)

3. **Elevación (Lift):** *¿Es una relación real o una coincidencia?*

- **Pregunta de Negocio:** ¿La compra de pan realmente impulsa la compra de leche, o la gente simplemente compra mucha leche de todos modos?

- **Explicación Directa:** El Lift es la métrica más importante porque nos dice si una regla es realmente significativa o si es solo una coincidencia. Compara la probabilidad de que dos productos se compren juntos con la probabilidad de que se compren si fueran eventos independientes. Corrige los posibles engaños de la confianza.

- **Fórmula:**

$$\text{Lift}(A \rightarrow B) = \frac{\text{Confianza}(A \rightarrow B)}{\text{Soporte}(B)}$$

- **Ejemplo:**

  - Imaginemos la famosa regla {pañales} -> {cerveza}. Supongamos que la confianza de {pañales} -> {cerveza} es del 20%. Parece baja. Pero, ¿qué pasa si solo el 2% de todos los clientes compran cerveza?
  
    El Lift se calcularía como Confianza({pañales} -> {cerveza}) / Soporte({cerveza}) = 20% / 2% = 10.

    Un Lift de 10 es altísimo. Significa que los clientes que compran pañales son 10 veces más propensos a comprar cerveza que un cliente promedio. Esa es una asociación muy fuerte y útil.

- **Interpretación del Lift:**

  - Lift > 1: La compra del producto A aumenta la probabilidad de que se compre el producto B. Son productos complementarios y la regla es interesante.

  - Lift = 1: La compra de A no tiene ningún efecto en la compra de B. Son productos independientes y la regla no aporta valor.

  - Lift < 1: La compra de A disminuye la probabilidad de que se compre el producto B. Son productos sustitutos (ej. si compran Coca-Cola, es menos probable que compren Pepsi).

## Un Ejemplo Intuitivo

Antes de usar librerías complejas, vamos a calcular las métricas a mano con un mini-dataset. Esto es crucial para construir la intuición de lo que el algoritmo hará por nosotros.

Imaginemos que tenemos 5 transacciones en nuestro pequeño café:

In [ ]:
# Dataset de transacciones de un café
transacciones = [['cafe', 'pan', 'leche'],
               ['cafe', 'azucar'],
               ['cafe', 'pan', 'azucar'],
               ['leche', 'pan'],
               ['cafe', 'leche', 'pan', 'azucar']]

total_transacciones = len(transacciones)
print(f"Tenemos un total de {total_transacciones} transacciones.")

### 1.1 Calculando el Soporte

Queremos encontrar el soporte para el itemset `{cafe, pan}`. Es decir, ¿en qué proporción de las transacciones aparecen el café y el pan juntos?

In [ ]:
# Contemos manualmente
frecuencia_cafe_pan = 0
for t in transacciones:
    if 'cafe' in t and 'pan' in t:
        frecuencia_cafe_pan += 1

soporte_cafe_pan = frecuencia_cafe_pan / total_transacciones

print(f"El itemset {{'cafe', 'pan'}} aparece en {frecuencia_cafe_pan} transacciones.")
print(f"El soporte de {{'cafe', 'pan'}} es: {frecuencia_cafe_pan}/{total_transacciones} = {soporte_cafe_pan:.2f}")

### 1.2 Calculando la Confianza

Ahora, evaluemos la regla `{cafe} -> {pan}`. Si alguien compra café, ¿qué tan probable es que también compre pan?

Necesitamos dos cosas: el soporte de `{cafe, pan}` (que ya tenemos) y el soporte de `{cafe}`.

In [ ]:
# Primero, soporte de {cafe}
frecuencia_cafe = 0
for t in transacciones:
    if 'cafe' in t:
        frecuencia_cafe += 1

soporte_cafe = frecuencia_cafe / total_transacciones

print(f"El itemset {{'cafe'}} aparece en {frecuencia_cafe} transacciones.")
print(f"El soporte de {{'cafe'}} es: {frecuencia_cafe}/{total_transacciones} = {soporte_cafe:.2f}")

In [ ]:
# Ahora calculamos la confianza
confianza_cafe_hacia_pan = soporte_cafe_pan / soporte_cafe

print(f"\nLa confianza de la regla {{'cafe'}} -> {{'pan'}} es: {confianza_cafe_hacia_pan:.2f}")
print(f"Interpretación: El 75% de las veces que alguien compra café, también compra pan.")

### 1.3 Calculando el Lift (Elevación)

Finalmente, ¿la regla `{cafe} -> {pan}` es realmente fuerte, o la gente simplemente compra mucho pan independientemente de si compra café?

Necesitamos el soporte de `{pan}`.

In [ ]:
# Primero, soporte de {pan}
frecuencia_pan = 0
for t in transacciones:
    if 'pan' in t:
        frecuencia_pan += 1

soporte_pan = frecuencia_pan / total_transacciones

print(f"El itemset {{'pan'}} aparece en {frecuencia_pan} transacciones.")
print(f"El soporte de {{'pan'}} es: {frecuencia_pan}/{total_transacciones} = {soporte_pan:.2f}")

In [ ]:
# Ahora calculamos el lift
lift_cafe_pan = confianza_cafe_hacia_pan / soporte_pan

print(f"\nEl Lift de la regla {{'cafe'}} -> {{'pan'}} es: {lift_cafe_pan:.2f}")
print(f"Interpretación: Como el Lift es 0.94 (menor a 1), la presencia de café en la cesta en realidad disminuye ligeramente la probabilidad de comprar pan. La regla no es tan fuerte como parecía con la confianza.")

## Aplicación Real - Análisis de Cesta de un Supermercado

Ahora que entendemos la mecánica, vamos a aplicar el algoritmo a un dataset real. Usaremos el dataset **"Market Basket Optimisation"** de Kaggle, que contiene 7.501 transacciones de un supermercado francés.

**Librerías que usaremos:**
* **Pandas:** Para la manipulación de datos.
* **MLxtend (Machine Learning Extensions):** Es la herramienta estándar y recomendada en el ecosistema de Python para el análisis de reglas de asociación. Fue creada para complementar a scikit-learn con algoritmos y herramientas útiles que no están en el núcleo principal de la librería. Su implementación de Apriori es robusta, eficiente y la más utilizada.

In [ ]:
# Instalamos la librería si no está presente
#!pip install mlxtend

In [ ]:
import os
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

## Mejorar visualización de dataframes

In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

### 2.1 Carga y Preprocesamiento de Datos

El dataset original está en un formato "ancho", donde cada fila es una transacción y cada columna un ítem de esa transacción. El algoritmo Apriori necesita un formato diferente: una matriz booleana donde cada fila es una transacción y cada columna representa un producto único de la tienda. El valor será `True` o `1` si el producto está en la transacción, y `False` o `0` si no lo está.

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/2025_ii_curso_ia_economia/datasets'

In [ ]:
# Para establecer el directorio de los archivos
os.chdir(path)

In [ ]:
df = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
df

In [ ]:
print("Dimensiones del dataset:", df.shape)

In [ ]:
# Paso 1: Convertir el DataFrame de pandas a una lista de listas (transacciones)
# Es importante manejar los valores 'NaN' que indican que no hay más productos en esa transacción
transacciones_lista = []
for i in range(0, df.shape[0]):
    # Para la fila i del DataFrame, crea una lista con cada valor de la fila,
    # pero solo si ese valor no es 'nan', y luego agrega esa nueva lista a transacciones_lista.
    transacciones_lista.append([str(df.values[i,j]) for j in range(0, df.shape[1]) if str(df.values[i,j]) != 'nan'])

# Veamos las primeras 5 transacciones
transacciones_lista[:5]

In [ ]:
# Paso 2: Usar TransactionEncoder para transformar la lista en una matriz booleana
# Creamos una instancia de TransactionEncoder.
te = TransactionEncoder()
# Aplicamos el código a nuestra lista de transacciones
te_ary = te.fit(transacciones_lista).transform(transacciones_lista)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

print("Dimensiones del dataset transformado:", df_encoded.shape)
df_encoded.head()

### 2.2 Aplicando el Algoritmo Apriori

El proceso tiene dos fases:
1.  **Generar `frequent_itemsets`:** Encontrar todas las combinaciones de productos cuyo **soporte** sea mayor a un umbral mínimo (`min_support`). Este umbral es clave: si es muy bajo, tendremos demasiadas combinaciones irrelevantes; si es muy alto, podríamos perder patrones interesantes.
2.  **Generar `association_rules`:** A partir de los itemsets frecuentes, calcular las métricas (confianza, lift) para generar las reglas finales, filtrando por un umbral mínimo de interés (ej. `lift > 1`).

#### Fase 1: Encontrar itemsets frecuentes

El objetivo acá es analizar todas las transacciones y devolver una lista de todas las combinaciones de productos ((ítems que aparecen solos, en pares, en tríos, etc.)) que superan un umbral de popularidad mínimo.

In [ ]:
# Empezaremos con un soporte mínimo del 1% (0.01)
# Esto significa que solo consideraremos combinaciones de productos que aparezcan en al menos el 1%
# de todas las transacciones.
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False).head(10)

In [ ]:
frequent_itemsets.shape

**Nota: **

En cuanto al soporte mínimo, la práctica más común es empezar con un valor relativamente alto y disminuirlo de forma iterativa hasta encontrar un balance entre la cantidad de resultados y su relevancia.

Un buen rango para empezar suele ser entre 0,05 (5%) y 0.001 (0.1%).

- Si min_support es muy alto (ej. 20%): Solo encontrarás las combinaciones más obvias y populares (pan, leche, huevos). El algoritmo será muy rápido, pero te perderás de patrones de nicho que suelen ser los más valiosos.

- Si min_support es muy bajo (ej. 0.0001%): El algoritmo puede tardar una eternidad o agotar la memoria de tu computador. Se producirá una "explosión combinatoria" y generarás miles de reglas, la mayoría de las cuales serán ruido o coincidencias sin valor comercial.

#### Fase 2: Generar las reglas de asociación

In [ ]:
# Usaremos 'lift' como métrica y un umbral de 1.2 para asegurar que las asociaciones sean positivas y no triviales.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
# Mostremos las 15 reglas más fuertes, ordenadas por Lift y Confianza
rules = rules[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'confidence', 'lift']]
rules.sort_values(['lift', 'confidence'], ascending=[False, False]).head(15)

In [ ]:
rules.shape

### 2.3 Interpretación de Resultados y Conclusiones Económicas

La tabla anterior es el resultado final de nuestro análisis. Ahora, el verdadero valor está en traducirla a insights de negocio. Analicemos un par de reglas interesantes:

**Regla: `{ground beef}` -> `{herb & pepper}`**
* **Soporte (antecedents):** 0.1. La carne molida se compra en ~10% de las transacciones.
* **Soporte (consequents):** 0.05. Las hierbas y la pimienta se compran juntas en ~5% de las transacciones.
* **Confianza:** 0.16. Si un cliente compra carne molida, hay un 16% de probabilidad de que también compre  hierbas y pimienta.
* **Lift:** 3.3. ¡Este es el dato clave! Los clientes que compran carne son **3,29 veces más propensos** a comprar hierbas y pimienta que un cliente promedio.
* **Conclusión Económica/De Negocio:** Esta fuerte asociación sugiere un patrón de cocina específico, quizás para una receta en particular francesa.
    * **Acción 1 (Cross-selling):** Colocar las especias cerca de la sección de carnes.
    * **Acción 2 (Marketing):** Crear una campaña de marketing con una receta que use estos tres ingredientes, ofreciendo un pequeño descuento si se compran juntos.

## Ejercicio

1.  Filtra las reglas para encontrar todas aquellas donde el consecuente (el producto comprado 'gracias a' otros) sea **'french fries'**.
2.  De esas reglas, identifica cuál tiene el **Lift más alto**.
3.  **Interpreta en una frase** qué significa esa regla para el negocio.


In [ ]:
### TU CÓDIGO AQUÍ ###

# 1. Filtrar las reglas


# 2. Ordenar para encontrar el Lift más alto


# 3. Escribe tu interpretación como un comentario